# The Battle of Neighborhoods: New York vs. Toronto
#### by Olga Fernandez Solivan


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Problem Statement and Stakeholders](#problem)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

#### A friend of mine received two job offers and asked me to performed an analysis of the venues of these two locations in order to make an informaed decision about two job offers. These jobs are located in Toronto and New York, and my friend would need to relocate with his family.  However, every Friday they order pizza for pick-up and watch a movie at home. His family considers this family activity a tradition that they would like to keet alive after relocation; thus my analysis will be focus on searching for the best pizza restaurants between these two Cities: New York and Toronto. 

## Problem Statement and Stakeholders <a name="problem"></a>

#### This analysis solves the problem of ​relocating for a job opportunity. Relocating can change someone's life and that person's entire family. During my career, I've relocated three times across the United States to obtain better opportunities within the company that I want to work. However, every relocation has come with trade-offs that other employees may not want for their lives. For example, currently, I have a friend that is married and has kids. His problem is that he doesn't want to relocate to a place where his family traditions will be impacted, and one of those activities that they want to keep is to be able to pick-up pizza from a pizza place that has good pizza. This problem that may seem trivial is so essential because it impacts worker's and families' lives, and I care about my friend and his family. Thus, the target audience for this analysis is my friend and his family. Please refer to my drafted notebook in github for further details.

## Data <a name="data"></a>

#### For this analysis, I will be parsing a Wikipedia page containing the City of Toronto's postal codes by boroughs and neighborhoods. Also I will be uploading a JSON file with New York City geographical data. Both of them have neighborhoods, boroughs, and geographical data (i.e., longitude and latitude coordinates) that I will be using to obtain a list of venues withing a geographical area from Foursquare. The Foursquare dataset contains venue geographical information and tips that I will be using to search within the City of Toronto and New York City. Specifically, I will be looking for pizza places with the best ratings. Please refer to my drafted notebook in github for further details.

## Methodology <a name="methodology"></a>

#### I'll be performing in this notebook a comparison between New York and Toronto. For this analysis, I will perform segmentation and clustering of these two Cities. To accomplish these tasks, I will use a new Jupyter notebook to contruct my code and I will scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to identify the major neighborhoods of these two citites.  I addition I will use an Fourquare API App to find the most recommended pizza restaurants. Once I transform these data from the wikipedia page into a pandas dataframe of the Postal, Code, Borough, and Neighborhood, through clustering I will identify the neighborhoods in the city of Toronto.  To identify the neighborhoods int he City of New York I will use a jason file that was already downloaded into the server. (i.e. 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json)

## Analysis <a name="analysis"></a>

#### Let's start with importing the necessary libraries and the installations of geopy and folium.

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

In [4]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

In [5]:
import json # library to handle JSON files

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [6]:
! pip install folium==0.5.0
import folium # plotting library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
import requests # library to handle requests

In [8]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [9]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [10]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geopy --yes 

Solving environment: done

# All requested packages already installed.



In [11]:
!pip install folium
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


Define Foursquare Credentials and Version
Make sure that you have created a Foursquare developer account and have your credentials handy
To obtain access token follow these steps.


Go to your "App Settings" page on the developer console of Foursquare.com
Set the "Redirect URL" under "Web Addresses" to https://www.google.com
Paste and enter the following url in your web browser (replace YOUR_CLIENT_ID with your actual client id): https://foursquare.com/oauth2/authenticate?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=https://www.google.com

This should redirect you to a google page requesting permission to make the connection.

Accept and then look at the url of your web browser (take note at the CODE part of the url to use in step 5)
It should look like https://www.google.com/?code=CODE

Copy the code value from the previous step.
Paste and enter the following into your web browser (replace placeholders with actual values): https://foursquare.com/oauth2/access_token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_CLIENT_SECRET&grant_type=authorization_code&redirect_uri=https://www.google.com&code=CODE.

When you paste the link , This should lead you to a page that gives you your access token.

In [12]:
CLIENT_ID = 'WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL' # your Foursquare ID
CLIENT_SECRET = 'VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN' # your Foursquare Secret
ACCESS_TOKEN = 'UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL
CLIENT_SECRET:VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN


In [13]:
#### My friend will be going to New York City to explore the City and its neighborhoods.  He will be staying at the Park Central Hotel. First, I need to find the the Park Central Hotel's latitude and longitude coordinates.

address = '870 7th Ave, New York, NY 10019'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7646446 -73.9812106


#### Search for a specific venue category

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**
`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


#### For lunch time, is planning to have Pizza. Therefore, I'll define the following query to search for Pizza that is within 500 metres from the Park Central Hotel.

In [14]:
search_query = 'Pizza'
radius = 500
print(search_query + ' .... Got it!')

Pizza .... Got it!


#### Define the corresponding URL using the template provided above: 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)



In [15]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&ll=40.7646446,-73.9812106&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604&query=Pizza&radius=500&limit=30'

#### Send the GET Request and examine the results


In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015b4e54c547e3804d266ae'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4cddadce7e2e236ab85d7a1b',
    'name': '99¢ Fresh Pizza',
    'location': {'address': '1723 Broadway',
     'crossStreet': 'btwn W 54th St & W 55th St',
     'lat': 40.76453737210592,
     'lng': -73.98238872801745,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.76453737210592,
       'lng': -73.98238872801745}],
     'distance': 100,
     'postalCode': '10019',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1723 Broadway (btwn W 54th St & W 55th St)',
      'New York, NY 10019',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_'

#### My friend will be going to New York City to explore the City and its neighborhoods.  He will be staying at the Park Central Hotel. First, I need to find the the Park Central Hotel's latitude and longitude coordinates.

address = '870 7th Ave, New York, NY 10019'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

#### Get relevant part of JSON and transform it into a _pandas_ dataframe


In [17]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4cddadce7e2e236ab85d7a1b,99¢ Fresh Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1612035301,False,1723 Broadway,btwn W 54th St & W 55th St,40.764537,-73.982389,"[{'label': 'display', 'lat': 40.76453737210592...",100,10019,US,New York,NY,United States,"[1723 Broadway (btwn W 54th St & W 55th St), N...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b91abc8f964a520bece33e3,Famous Original Ray's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1612035301,False,831 7th Ave,btwn 53rd & 54th St,40.763358,-73.981742,"[{'label': 'display', 'lat': 40.76335812438516...",150,10019,US,New York,NY,United States,"[831 7th Ave (btwn 53rd & 54th St), New York, ...",290505,https://www.seamless.com/menu/famous-original-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,73145054
2,4b4c704bf964a520feb226e3,Famous Original Ray's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1612035301,False,736 7th Ave,btwn 48th & 49th St.,40.760438,-73.983929,"[{'label': 'display', 'lat': 40.76043774660137...",521,10019,US,New York,NY,United States,"[736 7th Ave (btwn 48th & 49th St.), New York,...",292848,https://www.seamless.com/menu/famous-original-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
3,4b272785f964a520f98424e3,Famous Original Ray's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1612035301,False,1710 Broadway,btwn 54th & 55th St.,40.764253,-73.982552,"[{'label': 'display', 'lat': 40.76425259026843...",121,10019,US,New York,NY,United States,"[1710 Broadway (btwn 54th & 55th St.), New Yor...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4aa88507f964a5208a5120e3,Famous Amadeus Pizza - Hell's Kitchen,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1612035301,False,840 8th Ave,NaN,40.762357,-73.985667,"[{'label': 'display', 'lat': 40.76235730570203...",453,10019,US,New York,NY,United States,"[840 8th Ave, New York, NY 10019, United States]",65046,https://www.seamless.com/menu/amadeus-pizza-84...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,47491715


#### Define information of interest and filter dataframe

In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,99¢ Fresh Pizza,Pizza Place,1723 Broadway,btwn W 54th St & W 55th St,40.764537,-73.982389,"[{'label': 'display', 'lat': 40.76453737210592...",100,10019,US,New York,NY,United States,"[1723 Broadway (btwn W 54th St & W 55th St), N...",4cddadce7e2e236ab85d7a1b
1,Famous Original Ray's Pizza,Pizza Place,831 7th Ave,btwn 53rd & 54th St,40.763358,-73.981742,"[{'label': 'display', 'lat': 40.76335812438516...",150,10019,US,New York,NY,United States,"[831 7th Ave (btwn 53rd & 54th St), New York, ...",4b91abc8f964a520bece33e3
2,Famous Original Ray's Pizza,Pizza Place,736 7th Ave,btwn 48th & 49th St.,40.760438,-73.983929,"[{'label': 'display', 'lat': 40.76043774660137...",521,10019,US,New York,NY,United States,"[736 7th Ave (btwn 48th & 49th St.), New York,...",4b4c704bf964a520feb226e3
3,Famous Original Ray's Pizza,Pizza Place,1710 Broadway,btwn 54th & 55th St.,40.764253,-73.982552,"[{'label': 'display', 'lat': 40.76425259026843...",121,10019,US,New York,NY,United States,"[1710 Broadway (btwn 54th & 55th St.), New Yor...",4b272785f964a520f98424e3
4,Famous Amadeus Pizza - Hell's Kitchen,Pizza Place,840 8th Ave,NaN,40.762357,-73.985667,"[{'label': 'display', 'lat': 40.76235730570203...",453,10019,US,New York,NY,United States,"[840 8th Ave, New York, NY 10019, United States]",4aa88507f964a5208a5120e3
5,Sacco Pizza,Pizza Place,819 9th Ave,btwn 54th & 55th St.,40.766167,-73.987055,"[{'label': 'display', 'lat': 40.76616738438938...",521,10019,US,New York,NY,United States,"[819 9th Ave (btwn 54th & 55th St.), New York,...",4a823120f964a520d7f81fe3
6,The Pizza (TurnStyle),Pizza Place,Columbus Circle,NaN,40.766981,-73.982708,"[{'label': 'display', 'lat': 40.766981, 'lng':...",289,10019,US,New York,NY,United States,"[Columbus Circle, New York, NY 10019, United S...",572bdb0a498e05a2d3f34426
7,Mariella Pizza,Pizza Place,935 8th Ave,W 55th St,40.765597,-73.983492,"[{'label': 'display', 'lat': 40.76559659992028...",219,10019,US,New York,NY,United States,"[935 8th Ave (W 55th St), New York, NY 10019, ...",4af5c84cf964a520c9fc21e3
8,Columbus Circle Pizza,Pizza Place,10 Columbus Circle,NaN,40.767770,-73.982290,"[{'label': 'display', 'lat': 40.76777, 'lng': ...",359,10019,US,New York,NY,United States,"[10 Columbus Circle, New York, NY 10019, Unite...",5a5cf45f6cf01a6b3ce099b8
9,Angelo's Pizza,Pizza Place,1697 Broadway,53rd St.,40.763769,-73.982582,"[{'label': 'display', 'lat': 40.76376883602391...",151,10019,US,New York,NY,United States,"[1697 Broadway (53rd St.), New York, NY 10019,...",4b36a128f964a520823925e3


#### Next, let's visualize the Pizza restaurants that are nearby

In [19]:
dataframe_filtered.name

0                           99¢ Fresh Pizza
1               Famous Original Ray's Pizza
2               Famous Original Ray's Pizza
3               Famous Original Ray's Pizza
4     Famous Amadeus Pizza - Hell's Kitchen
5                               Sacco Pizza
6                     The Pizza (TurnStyle)
7                            Mariella Pizza
8                     Columbus Circle Pizza
9                            Angelo's Pizza
10                                 Pizzarte
11                              Pop's Pizza
12                           Champion Pizza
13                           Angelo's Pizza
14                              Gio's Pizza
15                             Gotham Pizza
16                        Pizza In The Park
17                   Abitino's Pizza & Cafe
18                         Radio City Pizza
19                    The Italian Pizza Co.
20         Teresa's Brick Oven Pizza & Cafe
21                     John's Pizza Parlour
22                   Mandrit's P

In [20]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**



#### Since I found multiple Pizza restaurants close to the Park Central Hotel, Let's explore the closest to the Hotel  -- _99 Cents Fresh Pizza_

In [21]:
venue_id = '4cddadce7e2e236ab85d7a1b' # ID of 99¢ Fresh Pizza
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4cddadce7e2e236ab85d7a1b?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604'

#### Get the venue overall rating

In [22]:
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.5


In [23]:
result['response']['venue']['tips']['count'] # of tips

57

#### This rating seems low.  So let's check the rating of the second closest pizza restaurant. 

In [24]:
venue_id = '4b272785f964a520f98424e3' # ID of Famous Original Ray's Pizza within 121 meters
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4b272785f964a520f98424e3?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604'

In [25]:
result['response']['venue']['tips']['count'] # of tips

57

#### Since the Rating of the Second closest venue is lower than the first one, let's check the rating of the third closest venue.

In [26]:
### Get the venue overall rating
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.3


In [27]:
result['response']['venue']['tips']['count']

29

In [28]:
venue_id = '4b91abc8f964a520bece33e3' # ID of Famous Original Ray's Pizza within 150 meters
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.9


In [29]:
result['response']['venue']['tips']['count'] # of tips

50

#### Since the first Pizza place has a slightly better rating, than the second and the third closest to the hotel let's explore it further. Let's get the tips for this venue.

In [30]:
venue_id = '4cddadce7e2e236ab85d7a1b' # ID of 99¢ Fresh Pizza
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4cddadce7e2e236ab85d7a1b?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604'

#### Send GET request for result

In [31]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4cddadce7e2e236ab85d7a1b',
 'name': '99¢ Fresh Pizza',
 'contact': {'phone': '2122452155', 'formattedPhone': '(212) 245-2155'},
 'location': {'address': '1723 Broadway',
  'crossStreet': 'btwn W 54th St & W 55th St',
  'lat': 40.76453737210592,
  'lng': -73.98238872801745,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.76453737210592,
    'lng': -73.98238872801745}],
  'postalCode': '10019',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['1723 Broadway (btwn W 54th St & W 55th St)',
   'New York, NY 10019',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/99-fresh-pizza/4cddadce7e2e236ab85d7a1b',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 57},


In [32]:
## Tips for Original Ray's Pizza within 150 meters
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015b5669b01da3a3ddbea00'},
 'response': {'tips': {'count': 57,
   'items': [{'id': '57d33913cd10a4a2d6251c74',
     'createdAt': 1473460499,
     'text': "Eatable and a good value. Not an authentic NYC slice. Cash only, there's an ATM inside. 2 slices and soda for $2.75. 50-cents gets a topping per slice, $2 for pies.",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/57d33913cd10a4a2d6251c74',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'firstName': 'Tommy', 'lastName': 'C', 'countryCode': 'US'},
     'authorInteractionType': 'meh'}]}}}

#### Get tips and list of associated features



In [33]:
tips = 'tips'
tips = results['response']['tips']['items']


tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips



In [34]:
pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips) # json normalize tips


In [35]:
# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName',]
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()



,text,agreeCount,disagreeCount,id,user.firstName,user.lastName
0,"Eatable and a good value. Not an authentic NYC slice. Cash only, there's an ATM inside. 2 slices and soda for $2.75. 50-cents gets a topping per slice, $2 for pies.",4,0,57d33913cd10a4a2d6251c74,Tommy,C


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips. Also, 'user.gender' and 'user.id' were not available. So, I can't search the Fourquare User with > `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

#### Explore a location

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**



#### My friend after lunch, wants to go for a walk to explore the area that's nearby the Park Central Hotel.

In [36]:
## Define the coordinate from the Park City Hotel (i.e. 40.7646446 -73.9812106)
latitude = 40.7646446
longitude = -73.9812106

#### Define URL

In [37]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&ll=40.7646446,-73.9812106&v=20180604&radius=500&limit=30'

#### Send GET request and examine results


In [38]:
import requests

In [40]:
results = requests.get(url).json()
'There are {} venues nearby the Park Central Hotel.'.format(len(results['response']['groups'][0]['items']))

'There are 30 venues nearby the Park Central Hotel.'

#### Get relevant part of JSON

In [41]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '3fd66200f964a520b6e71ee3',
  'name': 'Carnegie Hall',
  'location': {'address': '881 7th Ave',
   'crossStreet': 'at W 57th St',
   'lat': 40.7651249,
   'lng': -73.9799185,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.7651249,
     'lng': -73.9799185},
    {'label': 'entrance', 'lat': 40.765096, 'lng': -73.980151}],
   'distance': 121,
   'postalCode': '10019',
   'cc': 'US',
   'neighborhood': 'Theater District',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['881 7th Ave (at W 57th St)',
    'New York, NY 10019',
    'United States']},
  'categories': [{'id': '5032792091d4c4b30a586d5c',
    'name': 'Concert Hall',
    'pluralName': 'Concert Halls',
    'shortName': 'Concert Hall',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/m

#### Process JSON and convert it to a clean dataframe



In [42]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

<ipython-input-42-b7321e367fed>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(items) # flatten JSON


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Carnegie Hall,Concert Hall,881 7th Ave,at W 57th St,40.765125,-73.979918,"[{'label': 'display', 'lat': 40.7651249, 'lng': -73.9799185}, {'label': 'entrance', 'lat': 40.765096, 'lng': -73.980151}]",121,10019,US,Theater District,New York,NY,United States,"[881 7th Ave (at W 57th St), New York, NY 10019, United States]",3fd66200f964a520b6e71ee3
1,Weill Recital Hall at Carnegie Hall,Concert Hall,154 W 57th St,Seventh Ave,40.765367,-73.979528,"[{'label': 'display', 'lat': 40.765366517574364, 'lng': -73.97952845936908}]",163,10019,US,NaN,New York,NY,United States,"[154 W 57th St (Seventh Ave), New York, NY 10019, United States]",4ae39d67f964a520489721e3
2,Ed Sullivan Theater,Theater,1697 Broadway,at W 53rd St,40.763894,-73.982660,"[{'label': 'display', 'lat': 40.76389395128865, 'lng': -73.98266006802903}]",148,10019,US,NaN,New York,NY,United States,"[1697 Broadway (at W 53rd St), New York, NY 10019, United States]",49f5cd90f964a520b96b1fe3
3,Stern Auditorium / Perelman Stage at Carnegie Hall,Music Venue,NaN,NaN,40.765015,-73.979970,"[{'label': 'display', 'lat': 40.76501454610367, 'lng': -73.9799702167511}, {'label': 'routing', 'lat': 40.76531190460391, 'lng': -73.97972881793976}]",112,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",56f247a7498e93a37ea880db
4,Feinstein's/54 Below,Performing Arts Venue,254 W 54th St,btwn Broadway & 8th Ave,40.764668,-73.983645,"[{'label': 'display', 'lat': 40.764667524397495, 'lng': -73.98364473818286}]",205,10019,US,NaN,New York,NY,United States,"[254 W 54th St (btwn Broadway & 8th Ave), New York, NY 10019, United States]",4fc13376e4b0557038f203f2
5,Broadway Theatre,Theater,1681 Broadway,at W 53rd St,40.763645,-73.983304,"[{'label': 'display', 'lat': 40.763645106751696, 'lng': -73.98330370414483}]",208,10019,US,NaN,New York,NY,United States,"[1681 Broadway (at W 53rd St), New York, NY 10019, United States]",4a246c20f964a5201e7e1fe3
6,New York City Center,Theater,131 W 55th St,btwn 6th & 7th Ave,40.763927,-73.979503,"[{'label': 'display', 'lat': 40.763926987716815, 'lng': -73.97950288262027}, {'label': 'entrance', 'lat': 40.763728, 'lng': -73.979583}]",164,10019,US,NaN,New York,NY,United States,"[131 W 55th St (btwn 6th & 7th Ave), New York, NY 10019, United States]",4b53367af964a5208e9227e3
7,Zankel Hall at Carnegie Hall,Concert Hall,7th Ave and 57th St,57th St,40.765004,-73.980436,"[{'label': 'display', 'lat': 40.76500375725871, 'lng': -73.98043640043468}]",76,10019,US,NaN,New York,NY,United States,"[7th Ave and 57th St (57th St), New York, NY 10019, United States]",4ace86aff964a5200bd120e3
8,Souvlaki GR,Souvlaki Shop,162 W 56th St,btwn 6th Ave & 7th Ave,40.764600,-73.980194,"[{'label': 'display', 'lat': 40.7646, 'lng': -73.980194}, {'label': 'entrance', 'lat': 40.764639, 'lng': -73.980262}]",85,10019,US,NaN,New York,NY,United States,"[162 W 56th St (btwn 6th Ave & 7th Ave), New York, NY 10019, United States]",56048f23498e16b72f056fd9
9,Indian Accent,North Indian Restaurant,123 W 56th St,NaN,40.764127,-73.978787,"[{'label': 'display', 'lat': 40.76412724424148, 'lng': -73.97878651245028}, {'label': 'entrance', 'lat': 40.764201, 'lng': -73.978653}]",212,10019,US,NaN,New York,NY,United States,"[123 W 56th St, New York, NY 10019, United States]",56c55e43cd105142e6f95b94


#### Let's visualize these items on the map around our location

In [227]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Park City Hotel


# add Park City Hotel as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Park City Hotel',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

#### Explore Trending Venues

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

### Next, let's find out what are the venues trending (i.e. the places with the highest foot traffic).

In [44]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015b6897f624569572238a0'},
 'response': {'venues': []}}

### Check if any venues are trending at this time

In [45]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [46]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

### Visualize trending venues

In [228]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Park City Hotel


    # add Park City Hotel as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Park City Hotel',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [229]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in New York City</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

#### Dataset Download and Exploration



In [64]:
!pip install wget

In [75]:
!pip install curl

In [76]:
!curl -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

curl: Remote file name has no length!
curl: (23) Failed writing received data to disk/application
Data downloaded!


#### Load and explore the data



Next, let's load the data.



In [77]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [78]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the _features_ key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.



In [79]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [80]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranforming the data into a _pandas_ dataframe

The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.



In [81]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.



In [82]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.



In [83]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [84]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.



In [86]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Using geopy library to get the latitude and longitude values of New York City.



In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [87]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Map of New York with neighborhoods superimposed on top.



In [88]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Thanks to **Folium** I can zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.



Next, let's segment and cluster only the neighborhoods in Brooklyn. I have an ant that has living there for many years, and I think my friend would like to explore it. So let's slice the original dataframe and create a new dataframe of the Brooklyn data.

In [93]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


Let's get the geographical coordinates of Manhattan.



In [94]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.6501038, -73.9495823.


As we did with all of New York City, let's map out Brooklyn and it neighborhoods.



In [96]:
# create map of Brooklyn using latitude and longitude values
map_Brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Brooklyn)  
    
map_Brooklyn

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.



#### Defining Foursquare Credentials and Version



In [97]:
CLIENT_ID = 'WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL' # your Foursquare ID
CLIENT_SECRET = 'VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL
CLIENT_SECRET:VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN


#### Let's explore the first neighborhood in our dataframe.



Get the neighborhood's name.

In [98]:
Brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

Get the neighborhood's latitude and longitude values.



In [99]:
neighborhood_latitude = Brooklyn_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Brooklyn_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Brooklyn_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


#### Now, let's get the top 100 venues that are in Bay Ridge within a radius of 500 meters.



First, let's create the GET request URL. Name your URL **url**.



In [100]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
##create url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&v=20180605&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

Send the GET request and examine the resutls



In [101]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015c1fd4375590137641f85'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 83,
  'suggestedBounds': {'ne': {'lat': 40.63030106951066,
    'lng': -74.02470273356597},
   'sw': {'lat': 40.62130106051065, 'lng': -74.03653865351028}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b895827f964a5206c2d32e3',
       'name': 'Pilo Arts Day Spa and Salon',
       'location': {'address': '8412 3rd Ave',
        'lat': 40.62474788273414,
        'lng': -74.03059056940135,
        'labeledLatL

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [102]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.



In [105]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(15)

<ipython-input-105-a30b5e48884a>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Cocoa Grinder,Juice Bar,40.623967,-74.030863
3,Leo's Casa Calamari,Pizza Place,40.624200,-74.030931
4,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
5,Ho' Brah Taco Joint,Taco Place,40.622960,-74.031371
6,Georgian Dream Cafe and Bakery,Caucasian Restaurant,40.625586,-74.030196
7,The Bookmark Shoppe,Bookstore,40.624577,-74.030562
8,A.L.C. Italian Grocery,Grocery Store,40.623051,-74.031224
9,Brooklyn Market,Grocery Store,40.626939,-74.029948


And how many venues were returned by Foursquare?



In [106]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

83 venues were returned by Foursquare.


#### **Exploration of Neighborhoods in Brooklyn**



#### Let's create a function to repeat the same process to all the neighborhoods in Brooklyn



In [107]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _Brooklyn_venues_.

In [108]:
# type your answer here
## The correct answer is:
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


#### Let's check the size of the resulting dataframe



In [109]:
print(Brooklyn_venues.shape)
Brooklyn_venues.head()

(2719, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
4,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot


Let's check how many venues were returned for each neighborhood



In [110]:
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,48,48,48,48,48,48
Bay Ridge,83,83,83,83,83,83
Bedford Stuyvesant,29,29,29,29,29,29
Bensonhurst,34,34,34,34,34,34
Bergen Beach,6,6,6,6,6,6
Boerum Hill,86,86,86,86,86,86
Borough Park,22,22,22,22,22,22
Brighton Beach,42,42,42,42,42,42
Broadway Junction,20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues



In [111]:
print('There are {} uniques categories.'.format(len(Brooklyn_venues['Venue Category'].unique())))

There are 284 uniques categories.


#### Analyzing Each Neighborhood



In [114]:
# one hot encoding
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian 

And let's examine the new dataframe size.



In [115]:
Brooklyn_onehot.shape

(2719, 284)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [116]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian 

#### Let's confirm the new size

In [117]:
Brooklyn_grouped.shape

(70, 284)

#### Let's print each neighborhood along with the top 5 most common venues



In [133]:
num_top_venues = 5

for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0    Chinese Restaurant  0.08
1  Cantonese Restaurant  0.04
2            Donut Shop  0.04
3   Peruvian Restaurant  0.04
4      Sushi Restaurant  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.06
1                  Spa  0.06
2  American Restaurant  0.05
3          Pizza Place  0.05
4           Bagel Shop  0.04


----Bedford Stuyvesant----
           venue  freq
0    Coffee Shop  0.10
1    Pizza Place  0.07
2  Deli / Bodega  0.07
3           Café  0.07
4            Bar  0.07


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.21
1          Donut Shop  0.06
2  Italian Restaurant  0.06
3    Sushi Restaurant  0.06
4      Ice Cream Shop  0.06


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1          Playground  0.17
2  Athletics & Sports  0.17
3                Park  0.17
4      Baseball Field  0.17


----Boerum Hill----
                 venue  freq
0       

#### Let's put that into a _pandas_ dataframe

First, let's write a function to sort the venues in descending order.



In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.



In [135]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Sushi Restaurant,Peruvian Restaurant,Bubble Tea Shop,Pharmacy,Fast Food Restaurant,Donut Shop,Italian Restaurant,Gas Station,Cantonese Restaurant
1,Bay Ridge,Spa,Italian Restaurant,American Restaurant,Pizza Place,Bagel Shop,Bar,Greek Restaurant,Sandwich Place,Chinese Restaurant,Hookah Bar
2,Bedford Stuyvesant,Coffee Shop,Deli / Bodega,Café,Bar,Pizza Place,Cocktail Bar,Tiki Bar,Boutique,Gift Shop,Playground
3,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,Donut Shop,Pizza Place,Bakery,Flower Shop,Butcher,Shabu-Shabu Restaurant
4,Bergen Beach,Harbor / Marina,Baseball Field,Park,Playground,Athletics & Sports,Women's Store,Fast Food Restaurant,Factory,Falafel Restaurant,Farm


#### Clustering of Neighborhoods

Run _k_-means to cluster the neighborhood into 6 clusters.



In [136]:
# set number of clusters
kclusters = 6

Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 2, 0, 2, 1, 1, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.



In [137]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_data

# merge Brooklyn_grouped with Brooklyn_data to add latitude/longitude for each neighborhood
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2,Spa,Italian Restaurant,American Restaurant,Pizza Place,Bagel Shop,Bar,Greek Restaurant,Sandwich Place,Chinese Restaurant,Hookah Bar
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Chinese Restaurant,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,Donut Shop,Pizza Place,Bakery,Flower Shop,Butcher,Shabu-Shabu Restaurant
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Mexican Restaurant,Bank,Latin American Restaurant,Pizza Place,Bakery,Mobile Phone Shop,Fried Chicken Joint,Gym,Donut Shop,Record Shop
3,Brooklyn,Greenpoint,40.730201,-73.954241,2,Bar,Coffee Shop,Pizza Place,Cocktail Bar,Grocery Store,Yoga Studio,French Restaurant,Deli / Bodega,Mexican Restaurant,Record Shop
4,Brooklyn,Gravesend,40.595260,-73.973471,2,Pizza Place,Lounge,Italian Restaurant,Bus Station,Metro Station,Bakery,Martial Arts School,Liquor Store,Chinese Restaurant,Men's Store


Finally, let's visualize the resulting clusters

In [138]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examination of Clusters

Next let's determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1

In [139]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 0, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Bergen Beach,Harbor / Marina,Baseball Field,Park,Playground,Athletics & Sports,Women's Store,Fast Food Restaurant,Factory,Falafel Restaurant,Farm


#### Cluster 2

In [140]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 1, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Mexican Restaurant,Bank,Latin American Restaurant,Pizza Place,Bakery,Mobile Phone Shop,Fried Chicken Joint,Gym,Donut Shop,Record Shop
5,Brighton Beach,Restaurant,Russian Restaurant,Eastern European Restaurant,Gourmet Shop,Sushi Restaurant,Beach,Bank,Pharmacy,Mobile Phone Shop,Bookstore
10,East Flatbush,Chinese Restaurant,Print Shop,Liquor Store,Pharmacy,Park,Caribbean Restaurant,Fast Food Restaurant,Moving Target,Supermarket,Department Store
25,Cypress Hills,Fried Chicken Joint,Pizza Place,Donut Shop,Ice Cream Shop,Fast Food Restaurant,Latin American Restaurant,Bank,Dance Studio,Discount Store,Chinese Restaurant
26,East New York,Event Service,Fried Chicken Joint,Spanish Restaurant,Gym,Fast Food Restaurant,Metro Station,Caribbean Restaurant,Bus Station,Pizza Place,Plaza
27,Starrett City,Bus Station,Supermarket,Chinese Restaurant,Bus Stop,Donut Shop,Cosmetics Shop,Pharmacy,Pizza Place,American Restaurant,Caribbean Restaurant
29,Flatlands,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bar,Seafood Restaurant,Discount Store,Lounge,Video Store,Nightclub
33,Bath Beach,Chinese Restaurant,Sushi Restaurant,Peruvian Restaurant,Bubble Tea Shop,Pharmacy,Fast Food Restaurant,Donut Shop,Italian Restaurant,Gas Station,Cantonese Restaurant
34,Borough Park,Bank,Hotel,Pizza Place,Pharmacy,Café,Fast Food Restaurant,Coffee Shop,Restaurant,Chinese Restaurant,Sandwich Place
44,City Line,Donut Shop,Fried Chicken Joint,Pharmacy,Grocery Store,Shoe Store,Mobile Phone Shop,Bus Stop,Mexican Restaurant,South American Restaurant,Food


#### Cluster 3

In [141]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 2, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,American Restaurant,Pizza Place,Bagel Shop,Bar,Greek Restaurant,Sandwich Place,Chinese Restaurant,Hookah Bar
1,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,Donut Shop,Pizza Place,Bakery,Flower Shop,Butcher,Shabu-Shabu Restaurant
3,Greenpoint,Bar,Coffee Shop,Pizza Place,Cocktail Bar,Grocery Store,Yoga Studio,French Restaurant,Deli / Bodega,Mexican Restaurant,Record Shop
4,Gravesend,Pizza Place,Lounge,Italian Restaurant,Bus Station,Metro Station,Bakery,Martial Arts School,Liquor Store,Chinese Restaurant,Men's Store
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Creperie,Buffet,Café,Sandwich Place,Russian Restaurant,Restaurant,Clothing Store,Yoga Studio
7,Manhattan Terrace,Pizza Place,Ice Cream Shop,Donut Shop,Falafel Restaurant,Jazz Club,Grocery Store,Coffee Shop,Bank,Bakery,Bagel Shop
8,Flatbush,Deli / Bodega,Bakery,Pharmacy,Coffee Shop,Caribbean Restaurant,Mexican Restaurant,Pizza Place,Donut Shop,Chinese Restaurant,Sandwich Place
9,Crown Heights,Pizza Place,Café,Museum,Supermarket,Burger Joint,Candy Store,Grocery Store,Electronics Store,Bakery,Bagel Shop
11,Kensington,Thai Restaurant,Grocery Store,Ice Cream Shop,Pizza Place,Sandwich Place,Coffee Shop,Gas Station,Music Venue,Spa,Department Store
12,Windsor Terrace,Grocery Store,Plaza,Park,Diner,Food Truck,Wine Shop,Beer Store,Chinese Restaurant,Bookstore,Bar


#### Cluster 4

In [142]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Madison,Bagel Shop,Restaurant,Dessert Shop,Pizza Place,Italian Restaurant,Pilates Studio,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market


#### Cluster 5

In [143]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Women's Store,Filipino Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop


#### Cluster 6

In [144]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 5, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Canarsie,Thai Restaurant,Asian Restaurant,Gym,Caribbean Restaurant,Bus Line,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
59,Paerdegat Basin,Gym,Asian Restaurant,Food,Bus Line,Auto Garage,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

Next, I will create a dataframe consisting of three columns: PostalCode, Borough, and Neighborhood, arranged the data from a wikipedia page as follows:
* Only work with the cells that have an assigned borough. 
* Ignore cells with a borough that is Not assigned. 
* If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough. 

Last, I will use the .shape method to print the number of rows in the dataframe.



#### Request to get the Wikipedia Webpage

In [145]:
Toronto_data = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Toronto_City_Data = requests.get(Toronto_data)

#### Convert table from html into a pandas dataframe

In [146]:
df_TCD_Original = pd.read_html(Toronto_City_Data.content, header=0)[0]
df_TCD_Original

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [147]:
df_TCD_Original.shape

(180, 3)

#### Drop rows with "Not assigned" in the Borough Columns¶

In [148]:
df_TCD_Clean_1 = df_TCD_Original[df_TCD_Original.Borough !='Not assigned'].reset_index(drop=True)
df_TCD_Clean_1.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### For cells with a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [149]:
#Locate the cells with a borough and a "Not assigned" neighborhood.
df_TCD_Clean_1.loc[df_TCD_Clean_1.Neighbourhood == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [150]:
#Replace the "Not assigned" with the Borough
df_TCD_Clean_2 = df_TCD_Clean_1.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_TCD_Clean_2

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Print Number of Rows with Shape Method

In [151]:
df_TCD_Clean_2.shape

(103, 3)

#### Download the geographical coordinates of each postal code: http://cocl.us/Geospatial_data¶

In [162]:
import wget
wget.download('https://cocl.us/Geospatial_data', 'TCCoord.csv')

'TCCoord.csv'

In [163]:
TC_Coord = pd.read_csv("TCCoord.csv", delimiter=",")
TC_Coord [0:5]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Add Coordinates to Pandas database

First check that both tables have the same shape

In [164]:
TC_Coord.shape

(103, 3)

Since the shapes of TC-Coord and df_TCD_Clean_2 are the same, I can join both tables and create a new database.

In [165]:
df_TCD_Clean_3 = df_TCD_Clean_2.join(TC_Coord.set_index('Postal Code'), on ='Postal Code')

In [166]:
df_TCD_Clean_3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Explore and cluster the neighborhoods in Toronto

Create a new database with the boroughs that include the word "Toronto"


In [167]:
df_TCD_Clean_4 = df_TCD_Clean_3[df_TCD_Clean_3['Borough'].str.contains('Toronto',regex=False)].reset_index(drop=True)
df_TCD_Clean_4

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",43.686412,-79.400049


#### Visualization of the Neighbourhoods in the City of Toronto

In [168]:
#Define latitude and longitude with geocoder for the City of Toronto City Hall
address='100 Queen St. W. Toronto, ON M5H 2N2' ##I found this address in this webpae: https://www.toronto.ca/home/contact-us/
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
print(location)

Toronto City Hall, 100, Queen Street West, Financial District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada


In [169]:
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

43.6536032 -79.38400547469666


In [170]:
#Create first map
TCN_map = folium.Map(location=[latitude, longitude], zoom_start=10)

#display map
TCN_map

In [171]:
#add a marker to identify the Toronto City Hall
folium.CircleMarker(
    [latitude,longitude],
    radius=10,
    color='red',
    popup='Toronto City Hall',
    fill=True,
    fill_color='blue',
    fill_opacy=0.6
).add_to(TCN_map)
TCN_map

In [172]:
#identify the Neigbourhoods with a blue marker
TCN_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat,lng,borough,neighbourhood in zip(df_TCD_Clean_4['Latitude'],df_TCD_Clean_4['Longitude'],df_TCD_Clean_4['Borough'],df_TCD_Clean_4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.7,
    parse_html=False).add_to(TCN_map)

TCN_map

Visually I was able to identify four clusters. Therefore, I'm going to run k-means to cluster the neighborhood into 5 clusters
    

In [173]:
# set number of clusters
kclusters = 4
Toronto_grouped_clustering=df_TCD_Clean_4.drop(['Postal Code','Borough','Neighbourhood'],1)

#run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

#List the cluster labels generated for each row in df_TCD_Clean_4.
kmeans.labels_

array([1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 1], dtype=int32)

Next, I'm going to create a new database with the clustering labels

In [174]:
df_TCD_Clean_4

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",43.686412,-79.400049


In [175]:
# create new database with clusters
df_TCD_Clean_4.insert(0,'Cluster Labels',kmeans.labels_)

In [176]:
df_TCD_Clean_4.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,1,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,1,M4M,East Toronto,Studio District,43.659526,-79.340923
4,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [177]:
# create map
TC_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_TCD_Clean_4['Latitude'], df_TCD_Clean_4['Longitude'], df_TCD_Clean_4['Neighbourhood'], df_TCD_Clean_4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(TC_map_clusters)
       
TC_map_clusters

After examining the TC_map_clusters, I identified four clusters.

In [178]:
df_TCD_Clean_4.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,1,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,1,M4M,East Toronto,Studio District,43.659526,-79.340923
4,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [179]:
df_TCD_Clean_4.loc[df_TCD_Clean_4['Cluster Labels'] == 0, df_TCD_Clean_4.columns[[1] + list(range(2, df_TCD_Clean_4.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
32,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
35,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


In [180]:
df_TCD_Clean_4.loc[df_TCD_Clean_4['Cluster Labels'] == 1, df_TCD_Clean_4.columns[[1] + list(range(2, df_TCD_Clean_4.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
38,M7Y,East Toronto,"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",43.662744,-79.321558


In [181]:
df_TCD_Clean_4.loc[df_TCD_Clean_4['Cluster Labels'] == 2, df_TCD_Clean_4.columns[[1] + list(range(2, df_TCD_Clean_4.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",43.686412,-79.400049
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936
23,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307


In [182]:
df_TCD_Clean_4.loc[df_TCD_Clean_4['Cluster Labels'] == 3, df_TCD_Clean_4.columns[[1] + list(range(2, df_TCD_Clean_4.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
12,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
13,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
16,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
18,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
19,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


#### In conclusion, I noticed that the City of Toronto has the following four Boroughs and that the first two letters of the Postal Codes for these Boroughs Start with M4 and end with M7:

|Cluster Number | Boroughs      	|Postal Codes-First 2 Letters|
|---------------|-------------------|----------------------------|
|           0	| West Toronto	    |  M6                        | 
|           1	| East Toronto	    |  M4, M7                    |
|           2	| Central Toronto	|  M4, M5                    |
|           3	| Downtown Toronto	|  M4, M5, M7                |


#### Next, I need to explore Downtown Toronto.

In [184]:
CLIENT_ID = 'WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL' # your Foursquare ID
CLIENT_SECRET = 'VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN' # your Foursquare Secret
ACCESS_TOKEN = 'UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL
CLIENT_SECRET:VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN


In [190]:
#### My friend will be going to Toronto City to explore the City and its neighborhoods.  He will be staying at the Hilton Hotel.

address = '145 Richmond St W, Toronto, ON M5H' ## To get the Hilton Hotel latitude and longitude coordinates

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.649938649999996 -79.38538037286145


#### Search for a specific venue category with Fourquare API

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**
`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


#### For lunch time, he is planning to have Pizza, of course!. Therefore, I'll define the following query to search for Pizza that is within 500 metres from the Hilton Hotel.

In [191]:
search_query = 'Pizza'
radius = 500
print(search_query + ' .... Got it!')

Pizza .... Got it!


#### Define the corresponding URL using the template provided above: 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)



In [192]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&ll=43.649938649999996,-79.38538037286145&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604&query=Pizza&radius=500&limit=30'

#### Send the GET Request and examine the results


In [193]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015d26699d3e66bf49d9274'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4b2d3727f964a52047d124e3',
    'name': 'Pizzaiolo Gourmet Pizza',
    'location': {'address': '270 Adelaide St W',
     'crossStreet': 'at John St',
     'lat': 43.64783852702828,
     'lng': -79.39029271145894,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64783852702828,
       'lng': -79.39029271145894}],
     'distance': 459,
     'postalCode': 'M5H 1X6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['270 Adelaide St W (at John St)',
      'Toronto ON M5H 1X6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
 

In [195]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4b2d3727f964a52047d124e3,Pizzaiolo Gourmet Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}]",v-1612042854,False,270 Adelaide St W,at John St,43.647839,-79.390293,"[{'label': 'display', 'lat': 43.64783852702828, 'lng': -79.39029271145894}]",459,M5H 1X6,CA,Toronto,ON,Canada,"[270 Adelaide St W (at John St), Toronto ON M5H 1X6, Canada]",NaN
1,5690363d498e0b868217831d,JZs Pizza,"[{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}]",v-1612042854,False,232 Wellington St W,NaN,43.645145,-79.389296,"[{'label': 'display', 'lat': 43.645145, 'lng': -79.389296}]",619,M5V 3W1,CA,Toronto,ON,Canada,"[232 Wellington St W, Toronto ON M5V 3W1, Canada]",NaN
2,5a9b3ef3d1a40244fc9e8373,The Good Son Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}]",v-1612042854,False,111 Richmond St w,NaN,43.650696,-79.384149,"[{'label': 'display', 'lat': 43.650696, 'lng': -79.384149}]",130,M5H 2G4,CA,Toronto,ON,Canada,"[111 Richmond St w, Toronto ON M5H 2G4, Canada]",NaN
3,5c789831180b91003915b599,Pizza Forno,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}]",v-1612042854,False,217 Adelaide St West,NaN,43.648290,-79.387505,"[{'label': 'display', 'lat': 43.64829, 'lng': -79.387505}]",250,M5H 1W7,CA,Toronto,ON,Canada,"[217 Adelaide St West, Toronto ON M5H 1W7, Canada]",Entertainment District
4,4b6b0234f964a52043ec2be3,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}]",v-1612042854,False,228 QUEEN STREET WEST,NaN,43.650107,-79.390002,"[{'label': 'display', 'lat': 43.65010669805269, 'lng': -79.39000182194306}]",372,M5V 1Z6,CA,Toronto,ON,Canada,"[228 QUEEN STREET WEST, Toronto ON M5V 1Z6, Canada]",NaN


#### Define information of interest and filter dataframe

In [196]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Pizzaiolo Gourmet Pizza,Pizza Place,270 Adelaide St W,at John St,43.647839,-79.390293,"[{'label': 'display', 'lat': 43.64783852702828, 'lng': -79.39029271145894}]",459,M5H 1X6,CA,Toronto,ON,Canada,"[270 Adelaide St W (at John St), Toronto ON M5H 1X6, Canada]",NaN,4b2d3727f964a52047d124e3
1,JZs Pizza,Italian Restaurant,232 Wellington St W,NaN,43.645145,-79.389296,"[{'label': 'display', 'lat': 43.645145, 'lng': -79.389296}]",619,M5V 3W1,CA,Toronto,ON,Canada,"[232 Wellington St W, Toronto ON M5V 3W1, Canada]",NaN,5690363d498e0b868217831d
2,The Good Son Pizza,Pizza Place,111 Richmond St w,NaN,43.650696,-79.384149,"[{'label': 'display', 'lat': 43.650696, 'lng': -79.384149}]",130,M5H 2G4,CA,Toronto,ON,Canada,"[111 Richmond St w, Toronto ON M5H 2G4, Canada]",NaN,5a9b3ef3d1a40244fc9e8373
3,Pizza Forno,Pizza Place,217 Adelaide St West,NaN,43.648290,-79.387505,"[{'label': 'display', 'lat': 43.64829, 'lng': -79.387505}]",250,M5H 1W7,CA,Toronto,ON,Canada,"[217 Adelaide St West, Toronto ON M5H 1W7, Canada]",Entertainment District,5c789831180b91003915b599
4,Pizza Pizza,Pizza Place,228 QUEEN STREET WEST,NaN,43.650107,-79.390002,"[{'label': 'display', 'lat': 43.65010669805269, 'lng': -79.39000182194306}]",372,M5V 1Z6,CA,Toronto,ON,Canada,"[228 QUEEN STREET WEST, Toronto ON M5V 1Z6, Canada]",NaN,4b6b0234f964a52043ec2be3
5,Pizzaiolo,Pizza Place,289 Dundas St,at McCaul St,43.654332,-79.391107,"[{'label': 'display', 'lat': 43.654332488625734, 'lng': -79.39110722018398}]",672,NaN,CA,Toronto,ON,Canada,"[289 Dundas St (at McCaul St), Toronto ON, Canada]",NaN,4b50edc9f964a5201c3927e3
6,Mama's Pizza,Pizza Place,NaN,NaN,43.648009,-79.382014,"[{'label': 'display', 'lat': 43.648009321614865, 'lng': -79.38201350873688}]",345,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4c4cc8e050710f47f771a232
7,Pizza Rustica Boutique,Italian Restaurant,232 wellington street west,John,43.645115,-79.389406,"[{'label': 'display', 'lat': 43.64511516109716, 'lng': -79.38940583644242}]",627,NaN,CA,Toronto,ON,Canada,"[232 wellington street west (John), Toronto ON, Canada]",NaN,4aeef7caf964a520ecd421e3
8,Classic italian style pizza food truck,Food Truck,CNE Midway,NaN,43.652144,-79.381118,"[{'label': 'display', 'lat': 43.652144, 'lng': -79.381118}]",422,NaN,CA,Toronto,ON,Canada,"[CNE Midway, Toronto ON, Canada]",NaN,4c787c9181bca0936180fa14
9,241 Pizza Delivery,Pizza Place,327 Queen St W,NaN,43.647954,-79.390683,"[{'label': 'display', 'lat': 43.64795421908404, 'lng': -79.3906831741333}]",480,M5V 2A4,CA,Toronto,ON,Canada,"[327 Queen St W, Toronto ON M5V 2A4, Canada]",NaN,4ee0f612f9f418e8e914d6f2


#### Next, let's visualize the Pizza restaurants that are nearby

In [197]:
dataframe_filtered.name

0                    Pizzaiolo Gourmet Pizza
1                                  JZs Pizza
2                         The Good Son Pizza
3                                Pizza Forno
4                                Pizza Pizza
5                                  Pizzaiolo
6                               Mama's Pizza
7                     Pizza Rustica Boutique
8     Classic italian style pizza food truck
9                         241 Pizza Delivery
10                          Freshslice Pizza
11                         Pizzeria Libretto
12                                 Pizzaiolo
13                                 Pizzaface
14                                 Pizzaiolo
Name: name, dtype: object

In [198]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### Exploring a Given Venue with Foursquare API

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**



#### Since I found multiple Pizza Places close to the Hilton Hotel, Let's explore the closest to the Hotel  -- The Good Son Pizza.

In [199]:
venue_id = '5a9b3ef3d1a40244fc9e8373' # ID of The Good Son Pizza
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/5a9b3ef3d1a40244fc9e8373?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604'

#### Get the venue overall rating

In [200]:
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [201]:
result['response']['venue']['tips']['count'] # of tips

1

#### This rating seems low and with only one tip, I will check the next closest Pizza Place. 

In [204]:
venue_id = '5462ac56498e128ccafe8fea' # ID of Pizzeria Libretto
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/5462ac56498e128ccafe8fea?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604'

In [205]:
result['response']['venue']['tips']['count'] # of tips

1

#### Since the Rating of the Second closest venue is lower than the first one, let's check the rating of the third closest venue (i.e. Pizza Forno).

In [206]:
### Get the venue overall rating
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.0


In [207]:
result['response']['venue']['tips']['count']

62

#### Bingo!!!! Pizza Borno has a high rating as well as a high number of responses. 

In [208]:
venue_id = '5c789831180b91003915b599' # ID of Pizza Forno
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/5c789831180b91003915b599?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&oauth_token=UG32CZ2U4JEPN5BVYUEWTFYW0DQE2ILRXLUZ33DB4XT1DFT2&v=20180604'

#### Send GET request for result

In [209]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '5c789831180b91003915b599',
 'name': 'Pizza Forno',
 'contact': {'twitter': 'thepizzaforno',
  'instagram': 'thepizzaforno',
  'facebook': '429392707534507',
  'facebookUsername': 'thepizzaforno',
  'facebookName': 'PizzaForno'},
 'location': {'address': '217 Adelaide St West',
  'lat': 43.64829,
  'lng': -79.387505,
  'labeledLatLngs': [{'label': 'display', 'lat': 43.64829, 'lng': -79.387505}],
  'postalCode': 'M5H 1W7',
  'cc': 'CA',
  'neighborhood': 'Entertainment District',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['217 Adelaide St West',
   'Toronto ON M5H 1W7',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/pizza-forno/5c789831180b91003915b599',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 's

In [210]:
## Tips for Pizza Forno
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015d89b44ede7784f6cd0cd'},
 'response': {'tips': {'count': 1,
   'items': [{'id': '5dab4a83c50172000a4201f7',
     'createdAt': 1571506819,
     'text': "Vending machine that cooks your pizza in 3 minutes. Quality is surprisingly decent but toppings are a bit sparse and the price isn't necessarily lower than the usual places",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5dab4a83c50172000a4201f7',
     'photo': {'id': '5dab4a83c50172000a4201f5',
      'createdAt': 1571506819,
      'source': {'name': 'Swarm for Android',
       'url': 'https://www.swarmapp.com'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/308951_5ei1KazTY6wCFXegDwOuSDCXllvzcNzs3pjsMbeKwpk.jpg',
      'width': 899,
      'height': 1900,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/308951_5ei1KazTY6wCFXegDwOuSDCXllvzcNzs3pjsMbeKwpk.jpg',
     'lang': 'en',
     'likes': {'count':

#### Get tips and list of associated features



In [211]:
tips = 'tips'
tips = results['response']['tips']['items']


tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips



In [212]:
pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips) # json normalize tips


In [213]:
# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName',]
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()



,text,agreeCount,disagreeCount,id,user.firstName,user.lastName
0,Vending machine that cooks your pizza in 3 minutes. Quality is surprisingly decent but toppings are a bit sparse and the price isn't necessarily lower than the usual places,0,0,5dab4a83c50172000a4201f7,Gabriel,S


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips. Also, 'user.gender' and 'user.id' were not available. So, I can't search the Fourquare User with > `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

#### Exploration of a location with Foursquare API

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**



#### My friend after lunch, wants to go for a walk to explore the area that's nearby Pizza Forno.

In [214]:
## Define the coordinate from Pizza Forno (i.e. 43.648290, -79.387505)
latitude = 43.648290
longitude = -79.387505

#### Define URL

In [215]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=WNW45UPOVV5VKYNULSPEDKY0LMHZVBPEJT5XDZRFRFXQ31IL&client_secret=VSDQCFFMPZMZKK0JYDRIIOUOR1CXOOHAG2KXYYJBV0RCOBVN&ll=43.64829,-79.387505&v=20180604&radius=500&limit=30'

#### Send GET request and examine results


In [216]:
import requests

In [218]:
results = requests.get(url).json()
'There are {} venues nearby Pizza Forno.'.format(len(results['response']['groups'][0]['items']))

'There are 30 venues nearby Pizza Forno.'

#### Get relevant part of JSON

In [219]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '529612de11d2ab526191ccc9',
  'name': 'Pai',
  'location': {'address': '18 Duncan St',
   'crossStreet': 'Adelaide and Duncan',
   'lat': 43.64792310735613,
   'lng': -79.38857932631602,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.64792310735613,
     'lng': -79.38857932631602}],
   'distance': 95,
   'postalCode': 'M5H 3G6',
   'cc': 'CA',
   'neighborhood': 'Entertainment District',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['18 Duncan St (Adelaide and Duncan)',
    'Toronto ON M5H 3G6',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d149941735',
    'name': 'Thai Restaurant',
    'pluralName': 'Thai Restaurants',
    'shortName': 'Thai',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/thai_',
     'suffix': '.png'},
    'primary': True}],


#### Process JSON and convert it to a clean dataframe



In [220]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

<ipython-input-220-b7321e367fed>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(items) # flatten JSON


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Pai,Thai Restaurant,18 Duncan St,Adelaide and Duncan,43.647923,-79.388579,"[{'label': 'display', 'lat': 43.64792310735613, 'lng': -79.38857932631602}]",95,M5H 3G6,CA,Entertainment District,Toronto,ON,Canada,"[18 Duncan St (Adelaide and Duncan), Toronto ON M5H 3G6, Canada]",529612de11d2ab526191ccc9
1,Byblos Toronto,Mediterranean Restaurant,11 Duncan Street,NaN,43.647615,-79.388381,"[{'label': 'display', 'lat': 43.647615054171766, 'lng': -79.3883810317138}]",103,M5V 3M2,CA,NaN,Toronto,ON,Canada,"[11 Duncan Street, Toronto ON M5V 3M2, Canada]",5321f4d9e4b07946702e6e08
2,Soho House Toronto,Speakeasy,192 Adelaide St W,at Simcoe St.,43.648734,-79.386541,"[{'label': 'display', 'lat': 43.64873430659662, 'lng': -79.38654117564838}]",92,M5H 0A4,CA,NaN,Toronto,ON,Canada,"[192 Adelaide St W (at Simcoe St.), Toronto ON M5H 0A4, Canada]",50322b6ae4b09116a296568c
3,JaBistro,Sushi Restaurant,222 Richmond St W,NaN,43.649687,-79.388090,"[{'label': 'display', 'lat': 43.64968685893743, 'lng': -79.38809001547467}]",162,M5V 1W4,CA,NaN,Toronto,ON,Canada,"[222 Richmond St W, Toronto ON M5V 1W4, Canada]",509bb871e4b09c7ac93f6642
4,Shangri-La Toronto,Hotel,188 University Ave.,at Adelaide St. W,43.649129,-79.386557,"[{'label': 'display', 'lat': 43.64912919417502, 'lng': -79.3865566853963}]",120,M5H 0A3,CA,NaN,Toronto,ON,Canada,"[188 University Ave. (at Adelaide St. W), Toronto ON M5H 0A3, Canada]",4e31b74252b131dcebb08743
5,Princess Of Wales Theatre,Theater,300 King St. W,at John St.,43.646543,-79.389100,"[{'label': 'display', 'lat': 43.64654315405365, 'lng': -79.38910030187289}]",233,M5V 1J2,CA,NaN,Toronto,ON,Canada,"[300 King St. W (at John St.), Toronto ON M5V 1J2, Canada]",4ad4c061f964a520aef720e3
6,Roy Thomson Hall,Concert Hall,60 Simcoe St,at King St W,43.646589,-79.385979,"[{'label': 'display', 'lat': 43.6465886970634, 'lng': -79.38597899374354}]",225,M5J 2H5,CA,NaN,Toronto,ON,Canada,"[60 Simcoe St (at King St W), Toronto ON M5J 2H5, Canada]",4ad4c061f964a520b0f720e3
7,Dark Horse Espresso Bar,Coffee Shop,125 John St,btwn Adelaide & Richmond St. W,43.648292,-79.390390,"[{'label': 'display', 'lat': 43.64829161679981, 'lng': -79.39039040155937}]",232,M5V 2E2,CA,NaN,Toronto,ON,Canada,"[125 John St (btwn Adelaide & Richmond St. W), Toronto ON M5V 2E2, Canada]",4f53aff7e4b07de37cecddf3
8,Lobby Lounge at the Shangri-La Toronto,Lounge,188 University Ave.,NaN,43.649155,-79.386546,"[{'label': 'display', 'lat': 43.64915499986854, 'lng': -79.38654574367817}]",123,M5H 0A3,CA,NaN,Toronto,ON,Canada,"[188 University Ave., Toronto ON M5H 0A3, Canada]",53856fa411d2061fc84a3d0a
9,Friendly Stranger - Cannabis Culture Shop,Smoke Shop,241 Queen St. W,at St. Patrick St.,43.650387,-79.388523,"[{'label': 'display', 'lat': 43.650386666114635, 'lng': -79.38852348995324}]",247,M5V 1Z4,CA,NaN,Toronto,ON,Canada,"[241 Queen St. W (at St. Patrick St.), Toronto ON M5V 1Z4, Canada]",4b7ed424f964a5208a0230e3


#### Let's visualize these items on the map around our location

In [221]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Pizza Forno


# add Pizza Forno as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Pizza Forno',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

#### Exploring Trending Venues with Foursquare API

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

#### Next, let's find out what are the venues trending (i.e. the places with the highest foot traffic).

In [222]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015da0932a2352a3161e542'},
 'response': {'venues': []}}

#### Check if any venues are trending at this time

In [223]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [224]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

#### Visualize trending venues

In [225]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Pizza Forno


    # add Pizza Forno as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Pizza Forno',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [226]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

## Results and Discussion <a name="results"></a>

This analysis revealed that both cities will be offering to my friends family various pizza places.  In New York, we explore the neighborhoods around Central Park and Brooklyn; and found numerous pizza places, just in Brooklyn we found 30 establishments.  Also in the City of Toronto we found several pizza places (14 in total) and one that specifically with a high number of tips and a significant high rating (Pizza Forno).  Furthermore, we explore the surrounding venues and foot traffic.  Unfortunately, we couldn’t draw any foot traffic results.  However, we still were able to identify numerous pizza places within 500 meters in both Cities. 

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify pizza places that could aid my stakeholders goals. I was able to gathered data-driven insights from segmenting and clustering the neighborhoods within the City of New York and Toronto. Furthermore I was able to leverage Foursquare data to identify borough, neighborhoods and its venues to support the recommendation to pursue bot job offers, since our analysis generated extensive collection of locations which satisfy some basic requirements regarding existing pizza places. The clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

In conclusion, based on this data analysis, I'm able to re-assure to my friend that both Cities have Pizza places that my fiend's family can attend to maintain their pizza night tradition. 